# Performance of sampled the state space of all the possible TensorFlow==2.1.0

The purpose of this notebook is to show performance of all possible combinations of the software stack for TensorFlow==2.1.0. These results obtained using Amun service explain some of the knowledge stored into Thoth that is used to provide Advises on AI software stacks.

## Inputs for Dataset created using Amun

All data have been gathered using [Amun Service](https://github.com/thoth-station/amun-api) and [Performance Indicators](https://github.com/thoth-station/performance) evaluated by Thoth Team.


## TensorFlow builds

Tensorflow builds have been created considering combinations of the following parameters:

**Software stacks and native dependencies**

All inspections use a combination of all stacks from the dependencies of TensorFlow in version 2.1.0. 


  * `upstream TensorFlow` - `tensorflow==2.1.0` available on PyPI (inspections prefixed with `tf`)

**OS images**

  * `rhel-8` 

**Python Interpreters**

  * `3.6` 
  
**Hardware**

No node pinning used, any hardware available on OCP is used. No GPU was used. 
Analysis across inspection run will show which hardware have been identified.

`Number of CPUs` used to run is selected a priori as input to Amun: 1

## Performance indicators
Performance Indicators (PI) used for performance analysis:

  * [matrix multiplication](https://github.com/thoth-station/performance/blob/master/tensorflow/matmul.py)

Each performance indicator was run `1 times` per inspection run (`batch size == 1`), performance indicators reported median of inspections to be further compared.

## Dataset content

Inspection specification, build logs, job logs, hardware information of the node where the performance indicator was run and the actual inspection job result are included in the dataset.

No buildtime errors spotted with the tested stack.

There are some runtime errors spotted with specific stack.


## Analysis

Results of performance are shown in terms of Elapsed time [ms].

The analysis performed in this notebook are defined as:

- Performance analysis across different Tf stacks (Python packages) (fixed Hardware, OS image, Python Interpreter, number of CPUs)


In [3]:
from thoth.report_processing.components.inspection import AmunInspections
from thoth.report_processing.components.inspection import AmunInspectionsSummary
from thoth.report_processing.components.inspection import AmunInspectionsStatistics
from thoth.report_processing.components.inspection import AmunInspectionsFailedSummary

inspection = AmunInspections()
inspection_runs_summary = AmunInspectionsSummary()
inspection_statistics = AmunInspectionsStatistics()
inspections_failed_sumary = AmunInspectionsFailedSummary()

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1500)
pd.options.plotting.backend = "plotly"  # Convert to matplotlib

In [ ]:
FILE_NAME = "thoth-performance-dataset-tf2.1.0-combinations.zip"
from thoth.report_processing.utils import extract_zip_file
extract_zip_file(FILE_NAME)

In [ ]:
inspections_identifiers = ["tf-dm-rw"]

In [ ]:
from pathlib import Path

current_path = Path.cwd()
inspection_runs = inspection.aggregate_thoth_inspections_results(
    inspections_identifiers=inspections_identifiers,
    is_local=True,
    repo_path=current_path.joinpath("inspections")
)

In [ ]:
processed_inspection_runs, failed_inspection_runs = inspection.process_inspection_runs(
    inspection_runs,
)

In [ ]:
inspections_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=processed_inspection_runs,
    include_statistics=True
)

In [7]:
inspections_failed_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=failed_inspection_runs
)

# Failed Inspections

In [8]:
inspections_failed_df.head()

,end_datetime,exit_code,hostname,script_sha256,start_datetime,stderr,inspection_document_id,identifier,specification_base,batch_size,hwinfo__cpu_features__flags,hwinfo__cpu_features__hz_actual,hwinfo__cpu_features__hz_actual_raw,hwinfo__cpu_features__hz_advertised,hwinfo__cpu_features__hz_advertised_raw,hwinfo__cpu_features__l1_data_cache_size,hwinfo__cpu_features__l1_instruction_cache_size,hwinfo__cpu_features__l2_cache_size,hwinfo__cpu_features__l3_cache_size,hwinfo__cpu_info__arch,hwinfo__cpu_info__arch_string_raw,hwinfo__cpu_info__bits,hwinfo__cpu_info__brand_raw,hwinfo__cpu_info__count,hwinfo__cpu_info__cpuinfo_version,hwinfo__cpu_info__cpuinfo_version_string,hwinfo__cpu_info__family,hwinfo__cpu_info__hz_actual_friendly,hwinfo__cpu_info__hz_advertised_friendly,hwinfo__cpu_info__l2_cache_associativity,hwinfo__cpu_info__l2_cache_line_size,hwinfo__cpu_info__model,hwinfo__cpu_info__stepping,hwinfo__cpu_info__vendor_id_raw,hwinfo__cpu_type__has_3dnow,hwinfo__cpu_type__has_3dnowext,hwinfo__cpu_type__has_Altivec,hwinfo__cpu_type__has_f00f_bug,hwinfo__cpu_type__has_fdiv_bug,hwinfo__cpu_type__has_mmx,hwinfo__cpu_type__has_sse,hwinfo__cpu_type__has_sse2,hwinfo__cpu_type__has_sse3,hwinfo__cpu_type__has_ssse3,hwinfo__cpu_type__is_32bit,hwinfo__cpu_type__is_64bit,hwinfo__cpu_type__is_AMD,hwinfo__cpu_type__is_AMD64,hwinfo__cpu_type__is_Alpha,hwinfo__cpu_type__is_Athlon64,hwinfo__cpu_type__is_AthlonHX,hwinfo__cpu_type__is_AthlonK6,hwinfo__cpu_type__is_AthlonK6_2,hwinfo__cpu_type__is_AthlonK6_3,hwinfo__cpu_type__is_AthlonK7,hwinfo__cpu_type__is_AthlonMP,hwinfo__cpu_type__is_Celeron,hwinfo__cpu_type__is_Core2,hwinfo__cpu_type__is_EV4,hwinfo__cpu_type__is_EV5,hwinfo__cpu_type__is_EV56,hwinfo__cpu_type__is_Hammer,hwinfo__cpu_type__is_Intel,hwinfo__cpu_type__is_Itanium,hwinfo__cpu_type__is_Nocona,hwinfo__cpu_type__is_Opteron,hwinfo__cpu_type__is_PCA56,hwinfo__cpu_type__is_Pentium,hwinfo__cpu_type__is_PentiumII,hwinfo__cpu_type__is_PentiumIII,hwinfo__cpu_type__is_PentiumIV,hwinfo__cpu_type__is_PentiumM,hwinfo__cpu_type__is_PentiumMMX,hwinfo__cpu_type__is_PentiumPro,hwinfo__cpu_type__is_Power,hwinfo__cpu_type__is_Power7,hwinfo__cpu_type__is_Power8,hwinfo__cpu_type__is_Power9,hwinfo__cpu_type__is_Prescott,hwinfo__cpu_type__is_XEON,hwinfo__cpu_type__is_Xeon,hwinfo__cpu_type__is_i386,hwinfo__cpu_type__is_i486,hwinfo__cpu_type__is_i586,hwinfo__cpu_type__is_i686,hwinfo__cpu_type__is_singleCPU,hwinfo__cpu_type__nbits,hwinfo__cpu_type__ncpus,hwinfo__cpu_type__not_impl,hwinfo__cpu_type__try_call,hwinfo__platform__architecture,hwinfo__platform__machine,hwinfo__platform__node,hwinfo__platform__platform,hwinfo__platform__processor,hwinfo__platform__release,hwinfo__platform__version,os_release__ansi_color,os_release__bug_report_url,os_release__cpe_name,os_release__home_url,os_release__id,os_release__id_like,os_release__name,os_release__platform_id,os_release__pretty_name,os_release__redhat_bugzilla_product,os_release__redhat_bugzilla_product_version,os_release__redhat_support_product,os_release__redhat_support_product_version,os_release__version,os_release__version_id,runtime_environment__cuda_version,runtime_environment__hardware__cpu_family,runtime_environment__hardware__cpu_model,runtime_environment__operating_system__name,runtime_environment__operating_system__version,runtime_environment__python_version,usage__ru_idrss,usage__ru_inblock,usage__ru_isrss,usage__ru_ixrss,usage__ru_majflt,usage__ru_maxrss,usage__ru_minflt,usage__ru_msgrcv,usage__ru_msgsnd,usage__ru_nivcsw,usage__ru_nsignals,usage__ru_nswap,usage__ru_nvcsw,usage__ru_oublock,usage__ru_stime,usage__ru_utime,requirements_locked___meta__sources,requirements_locked___meta__requires__python_version,requirements_locked___meta__hash__sha256,requirements_locked___meta__pipfile-spec,requirements_locked__default__tensorflow__version,requirements_locked__default__tensorflow__hashes,requirements_locked__default__tensorflow__index,requirements_locked__default__tensorflow-estimator__version,requirements_

In [9]:
inspections_failed_df['stderr'][0]

'2020-09-05 07:14:36.333589: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library \'libnvinfer.so.6\'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory\n2020-09-05 07:14:36.333811: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library \'libnvinfer_plugin.so.6\'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory\n2020-09-05 07:14:36.333844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.\n/opt/app-root/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.\n  from ._conv import register_

## Create inspection results summary

In [10]:
report_results_failed, _ = inspection_runs_summary.produce_summary_report(inspections_df=inspections_failed_df)

### Hardware

In [11]:
report_results_failed["hardware"]['platform'].head()

,hwinfo__platform__architecture,hwinfo__platform__machine,hwinfo__platform__node,hwinfo__platform__platform,hwinfo__platform__processor,hwinfo__platform__release,hwinfo__platform__version
0,nan,x86_64,inspection-tf-dm-rw-7068157c-4118133706,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
1,nan,x86_64,inspection-tf-dm-rw-8526bbb2-693839316,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
2,nan,x86_64,inspection-tf-dm-rw-2528dde4-3617758879,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
3,nan,x86_64,inspection-tf-dm-rw-08c26ccc-239457073,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
4,nan,x86_64,inspection-tf-dm-rw-1ff1966b-2739680822,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020


In [12]:
report_results_failed["hardware"]['processor']

,hwinfo__cpu_type__has_3dnow,hwinfo__cpu_type__has_3dnowext,hwinfo__cpu_type__has_Altivec,hwinfo__cpu_type__has_f00f_bug,hwinfo__cpu_type__has_fdiv_bug,hwinfo__cpu_type__has_mmx,hwinfo__cpu_type__has_sse,hwinfo__cpu_type__has_sse2,hwinfo__cpu_type__has_sse3,hwinfo__cpu_type__has_ssse3,hwinfo__cpu_type__is_32bit,hwinfo__cpu_type__is_64bit,hwinfo__cpu_type__is_AMD,hwinfo__cpu_type__is_AMD64,hwinfo__cpu_type__is_Alpha,hwinfo__cpu_type__is_Athlon64,hwinfo__cpu_type__is_AthlonHX,hwinfo__cpu_type__is_AthlonK6,hwinfo__cpu_type__is_AthlonK6_2,hwinfo__cpu_type__is_AthlonK6_3,hwinfo__cpu_type__is_AthlonK7,hwinfo__cpu_type__is_AthlonMP,hwinfo__cpu_type__is_Celeron,hwinfo__cpu_type__is_Core2,hwinfo__cpu_type__is_EV4,hwinfo__cpu_type__is_EV5,hwinfo__cpu_type__is_EV56,hwinfo__cpu_type__is_Hammer,hwinfo__cpu_type__is_Intel,hwinfo__cpu_type__is_Itanium,hwinfo__cpu_type__is_Nocona,hwinfo__cpu_type__is_Opteron,hwinfo__cpu_type__is_PCA56,hwinfo__cpu_type__is_Pentium,hwinfo__cpu_type__is_PentiumII,hwinfo__cpu_type__is_PentiumIII,hwinfo__cpu_type__is_PentiumIV,hwinfo__cpu_type__is_PentiumM,hwinfo__cpu_type__is_PentiumMMX,hwinfo__cpu_type__is_PentiumPro,hwinfo__cpu_type__is_Power,hwinfo__cpu_type__is_Power7,hwinfo__cpu_type__is_Power8,hwinfo__cpu_type__is_Power9,hwinfo__cpu_type__is_Prescott,hwinfo__cpu_type__is_XEON,hwinfo__cpu_type__is_Xeon,hwinfo__cpu_type__is_i386,hwinfo__cpu_type__is_i486,hwinfo__cpu_type__is_i586,hwinfo__cpu_type__is_i686,hwinfo__cpu_type__is_singleCPU
0,False,False,nan,nan,nan,True,True,True,True,True,False,True,False,False,nan,False,False,False,False,False,False,False,False,False,nan,nan,nan,False,True,nan,False,False,nan,False,False,False,False,False,False,False,nan,nan,nan,nan,False,True,True,nan,nan,False,True,False


In [13]:
report_results_failed["hardware"]['flags']

,flag__3dnowprefetch,flag__abm,flag__adx,flag__aes,flag__apic,flag__arat,flag__avx,flag__avx2,flag__avx512bw,flag__avx512cd,flag__avx512dq,flag__avx512f,flag__avx512vl,flag__bmi1,flag__bmi2,flag__clflush,flag__clflushopt,flag__clwb,flag__cmov,flag__constant_tsc,flag__cpuid,flag__cx16,flag__cx8,flag__de,flag__ept,flag__erms,flag__f16c,flag__flexpriority,flag__fma,flag__fpu,flag__fsgsbase,flag__fxsr,flag__hle,flag__hypervisor,flag__ibpb,flag__ibrs,flag__invpcid,flag__invpcid_single,flag__lahf_lm,flag__lm,flag__mca,flag__mce,flag__md_clear,flag__mmx,flag__movbe,flag__mpx,flag__msr,flag__mtrr,flag__nopl,flag__nx,flag__ospke,flag__osxsave,flag__pae,flag__pat,flag__pcid,flag__pclmulqdq,flag__pdpe1gb,flag__pge,flag__pku,flag__pni,flag__popcnt,flag__pse,flag__pse36,flag__pti,flag__rdrand,flag__rdrnd,flag__rdseed,flag__rdtscp,flag__rep_good,flag__rtm,flag__sep,flag__smap,flag__smep,flag__ss,flag__ssbd,flag__sse,flag__sse2,flag__sse4_1,flag__sse4_2,flag__ssse3,flag__stibp,flag__syscall,flag__tpr_shadow,flag__tsc,flag__tsc_adjust,flag__tsc_deadline_timer,flag__tsc_known_freq,flag__tscdeadline,flag__vme,flag__vmx,flag__vnmi,flag__vpid,flag__x2apic,flag__xgetbv1,flag__xsave,flag__xsavec,flag__xsaveopt,flag__xtopology
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [14]:
report_results_failed["hardware"]['ncpus']

,hwinfo__cpu_type__ncpus
0,64


In [15]:
report_results_failed["hardware"]['info']

,hwinfo__cpu_info__brand_raw,runtime_environment__cuda_version,runtime_environment__hardware__cpu_family,runtime_environment__hardware__cpu_model
0,"Intel Xeon Processor (Skylake, IBRS)",nan,6,85


### Operating System

In [16]:
report_results_failed["base_image"]['base_image']

,specification_base,os_release__name,os_release__version,os_release__version_id
0,quay.io/thoth-station/s2i-thoth-ubi8-py36,Red Hat Enterprise Linux,8.2 (Ootpa),8.2


In [17]:
report_results_failed["base_image"]['number_cpus_run']

,run__requests__cpu
0,1


### Performance Indicators

In [18]:
report_results_failed["pi"]['pi']

,script_sha256,batch_size
0,1ce2ae4cf0da06c8c55b99be8beb31cae7c6801bb99664...,1


### Software Stack

In [19]:
report_results_failed["software_stack"]['requirements_locked'].head()

,requirements_locked___meta__sources,requirements_locked___meta__requires__python_version,requirements_locked___meta__hash__sha256,requirements_locked___meta__pipfile-spec,requirements_locked__default__tensorflow__version,requirements_locked__default__tensorflow__index,requirements_locked__default__tensorflow-estimator__version,requirements_locked__default__tensorflow-estimator__index,requirements_locked__default__astor__version,requirements_locked__default__astor__index,requirements_locked__default__wheel__version,requirements_locked__default__wheel__index,requirements_locked__default__protobuf__version,requirements_locked__default__protobuf__index,requirements_locked__default__google-pasta__version,requirements_locked__default__google-pasta__index,requirements_locked__default__six__version,requirements_locked__default__six__index,requirements_locked__default__tensorboard__version,requirements_locked__default__tensorboard__index,requirements_locked__default__requests__version,requirements_locked__default__requests__index,requirements_locked__default__opt-einsum__version,requirements_locked__default__opt-einsum__index,requirements_locked__default__numpy__version,requirements_locked__default__numpy__index,requirements_locked__default__google-auth__version,requirements_locked__default__google-auth__index,requirements_locked__default__chardet__version,requirements_locked__default__chardet__index,requirements_locked__default__google-auth-oauthlib__version,requirements_locked__default__google-auth-oauthlib__index,requirements_locked__default__keras-applications__version,requirements_locked__default__keras-applications__index,requirements_locked__default__termcolor__version,requirements_locked__default__termcolor__index,requirements_locked__default__idna__version,requirements_locked__default__idna__index,requirements_locked__default__rsa__version,requirements_locked__default__rsa__index,requirements_locked__default__urllib3__version,requirements_locked__default__urllib3__index,requirements_locked__default__keras-preprocessing__version,requirements_locked__default__keras-preprocessing__index,requirements_locked__default__absl-py__version,requirements_locked__default__absl-py__index,requirements_locked__default__pyasn1-modules__version,requirements_locked__default__pyasn1-modules__index,requirements_locked__default__h5py__version,requirements_locked__default__h5py__index,requirements_locked__default__scipy__version,requirements_locked__default__scipy__index,requirements_locked__default__certifi__version,requirements_locked__default__certifi__index,requirements_locked__default__requests-oauthlib__version,requirements_locked__default__requests-oauthlib__index,requirements_locked__default__gast__version,requirements_locked__default__gast__index,requirements_locked__default__pyasn1__version,requirements_locked__default__pyasn1__index,requirements_locked__default__grpcio__version,requirements_locked__default__grpcio__index,requirements_locked__default__markdown__version,requirements_locked__default__markdown__index,requirements_locked__default__wrapt__version,requirements_locked__default__wrapt__index,requirements_locked__default__oauthlib__version,requirements_locked__default__oauthlib__index,requirements_locked__default__setuptools__version,requirements_locked__default__setuptools__index,requirements_locked__default__cachetools__version,requirements_locked__default__cachetools__index,requirements_locked__default__werkzeug__version,requirements_locked__default__werkzeug__index
0,nan,3.6,c9e7f586cc7f527664b35f4581b3d5421512489efb2c89...,6,==2.1.0,pypi-org,==2.1.0,pypi-org,==0.6.2,pypi-org,==0.31.1,pypi-org,==3.11.0,pypi-org,==0.1.6,pypi-org,==1.13.0,pypi-org,==2.1.0,pypi-org,==2.21.0,pypi-org,==3.0.1,pypi-org,==1.17.3,pypi-org,==1.6.3,pypi-org,==2.3.0,pypi-org,==0.4.1,pypi-org,==1.0.8,pypi-org,==1.1.0,pypi-org,==2.0,pypi-org,==3.2.2,pypi-org,==1.5,pypi-org,==1.1.0,pypi-org,==0.9.0,pypi-org,==0.2.7,pypi-org,==2.7.1,pypi-org,==1.4.1,pypi-org

In [20]:
python_packages_dataframe_failed, _ = inspection.create_python_package_df(inspections_df=inspections_failed_df)
python_packages_dataframe_failed.head()

,tensorflow,tensorflow-estimator,astor,wheel,protobuf,google-pasta,six,tensorboard,requests,opt-einsum,numpy,google-auth,chardet,google-auth-oauthlib,keras-applications,termcolor,idna,rsa,urllib3,keras-preprocessing,absl-py,pyasn1-modules,h5py,scipy,certifi,requests-oauthlib,gast,pyasn1,grpcio,markdown,wrapt,oauthlib,setuptools,cachetools,werkzeug
0,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-3.0.1-pypi-org,numpy-1.17.3-pypi-org,google-auth-1.6.3-pypi-org,chardet-2.3.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-2.0-pypi-org,rsa-3.2.2-pypi-org,urllib3-1.5-pypi-org,keras-preprocessing-1.1.0-pypi-org,absl-py-0.9.0-pypi-org,pyasn1-modules-0.2.7-pypi-org,h5py-2.7.1-pypi-org,scipy-1.4.1-pypi-org,certifi-2018.8.24-pypi-org,requests-oauthlib-1.0.0-pypi-org,gast-0.2.2-pypi-org,pyasn1-0.1.6-pypi-org,grpcio-1.24.0-pypi-org,markdown-3.1-pypi-org,wrapt-1.11.2-pypi-org,oauthlib-1.1.1-pypi-org,setuptools-21.2.1-pypi-org,cachetools-3.1.0-pypi-org,werkzeug-0.14.1-pypi-org
1,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-3.0.1-pypi-org,numpy-1.17.3-pypi-org,google-auth-1.6.3-pypi-org,chardet-2.3.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-2.0-pypi-org,rsa-3.2.2-pypi-org,urllib3-1.5-pypi-org,keras-preprocessing-1.1.0-pypi-org,absl-py-0.9.0-pypi-org,pyasn1-modules-0.2.7-pypi-org,h5py-2.7.1-pypi-org,scipy-1.4.1-pypi-org,certifi-2018.8.24-pypi-org,requests-oauthlib-1.0.0-pypi-org,gast-0.2.2-pypi-org,pyasn1-0.1.6-pypi-org,grpcio-1.24.0-pypi-org,markdown-3.0-pypi-org,wrapt-1.11.1-pypi-org,oauthlib-1.1.1-pypi-org,setuptools-36.1.0-pypi-org,cachetools-2.1.0-pypi-org,werkzeug-0.14-pypi-org
2,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-3.0.1-pypi-org,numpy-1.17.3-pypi-org,google-auth-1.6.3-pypi-org,chardet-2.3.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-2.0-pypi-org,rsa-3.2.2-pypi-org,urllib3-1.5-pypi-org,keras-preprocessing-1.1.0-pypi-org,absl-py-0.9.0-pypi-org,pyasn1-modules-0.2.1-pypi-org,h5py-2.8.0-pypi-org,scipy-1.4.1-pypi-org,certifi-2018.11.29-pypi-org,requests-oauthlib-0.8.0-pypi-org,gast-0.2.2-pypi-org,pyasn1-0.2.2-pypi-org,grpcio-1.19.0-pypi-org,markdown-2.6.11-pypi-org,wrapt-1.11.1-pypi-org,oauthlib-2.0.4-pypi-org,setuptools-32.1.1-pypi-org,cachetools-3.1.0-pypi-org,werkzeug-0.12.2-pypi-org
3,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-3.0.1-pypi-org,numpy-1.17.3-pypi-org,google-auth-1.6.3-pypi-org,chardet-2.3.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-2.0-pypi-org,rsa-3.2.2-pypi-org,urllib3-1.5-pypi-org,keras-preprocessing-1.1.0-pypi-org,absl-py-0.9.0-pypi-org,pyasn1-modules-0.2.1-pypi-org,h5py-2.8.0-pypi-org,scipy-1.4.1-pypi-org,certifi-2018.11.29-pypi-org,requests-oauthlib-0.8.0-pypi-org,gast-0.2.2-pypi-org,pyasn1-0.2.2-pypi-org,grpcio-1.19.0-pypi-org,markdown-2.6.11-pypi-org,wrapt-1.11.1-pypi-org,oauthlib-1.0.0-pypi-org,setuptools-32.1.1-pypi-org,cachetools-3.1.0-pypi-org,werkzeug-0.12.2-pypi-org
4,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,proto

# Successfull inspections

In [21]:
inspections_df.head()

,end_datetime,exit_code,hostname,script_sha256,start_datetime,stderr,inspection_document_id,identifier,specification_base,batch_size,hwinfo__cpu_features__flags,hwinfo__cpu_features__hz_actual,hwinfo__cpu_features__hz_actual_raw,hwinfo__cpu_features__hz_advertised,hwinfo__cpu_features__hz_advertised_raw,hwinfo__cpu_features__l1_data_cache_size,hwinfo__cpu_features__l1_instruction_cache_size,hwinfo__cpu_features__l2_cache_size,hwinfo__cpu_features__l3_cache_size,hwinfo__cpu_info__arch,hwinfo__cpu_info__arch_string_raw,hwinfo__cpu_info__bits,hwinfo__cpu_info__brand_raw,hwinfo__cpu_info__count,hwinfo__cpu_info__cpuinfo_version,hwinfo__cpu_info__cpuinfo_version_string,hwinfo__cpu_info__family,hwinfo__cpu_info__hz_actual_friendly,hwinfo__cpu_info__hz_advertised_friendly,hwinfo__cpu_info__l2_cache_associativity,hwinfo__cpu_info__l2_cache_line_size,hwinfo__cpu_info__model,hwinfo__cpu_info__stepping,hwinfo__cpu_info__vendor_id_raw,hwinfo__cpu_type__has_3dnow,hwinfo__cpu_type__has_3dnowext,hwinfo__cpu_type__has_Altivec,hwinfo__cpu_type__has_f00f_bug,hwinfo__cpu_type__has_fdiv_bug,hwinfo__cpu_type__has_mmx,hwinfo__cpu_type__has_sse,hwinfo__cpu_type__has_sse2,hwinfo__cpu_type__has_sse3,hwinfo__cpu_type__has_ssse3,hwinfo__cpu_type__is_32bit,hwinfo__cpu_type__is_64bit,hwinfo__cpu_type__is_AMD,hwinfo__cpu_type__is_AMD64,hwinfo__cpu_type__is_Alpha,hwinfo__cpu_type__is_Athlon64,hwinfo__cpu_type__is_AthlonHX,hwinfo__cpu_type__is_AthlonK6,hwinfo__cpu_type__is_AthlonK6_2,hwinfo__cpu_type__is_AthlonK6_3,hwinfo__cpu_type__is_AthlonK7,hwinfo__cpu_type__is_AthlonMP,hwinfo__cpu_type__is_Celeron,hwinfo__cpu_type__is_Core2,hwinfo__cpu_type__is_EV4,hwinfo__cpu_type__is_EV5,hwinfo__cpu_type__is_EV56,hwinfo__cpu_type__is_Hammer,hwinfo__cpu_type__is_Intel,hwinfo__cpu_type__is_Itanium,hwinfo__cpu_type__is_Nocona,hwinfo__cpu_type__is_Opteron,hwinfo__cpu_type__is_PCA56,hwinfo__cpu_type__is_Pentium,hwinfo__cpu_type__is_PentiumII,hwinfo__cpu_type__is_PentiumIII,hwinfo__cpu_type__is_PentiumIV,hwinfo__cpu_type__is_PentiumM,hwinfo__cpu_type__is_PentiumMMX,hwinfo__cpu_type__is_PentiumPro,hwinfo__cpu_type__is_Power,hwinfo__cpu_type__is_Power7,hwinfo__cpu_type__is_Power8,hwinfo__cpu_type__is_Power9,hwinfo__cpu_type__is_Prescott,hwinfo__cpu_type__is_XEON,hwinfo__cpu_type__is_Xeon,hwinfo__cpu_type__is_i386,hwinfo__cpu_type__is_i486,hwinfo__cpu_type__is_i586,hwinfo__cpu_type__is_i686,hwinfo__cpu_type__is_singleCPU,hwinfo__cpu_type__nbits,hwinfo__cpu_type__ncpus,hwinfo__cpu_type__not_impl,hwinfo__cpu_type__try_call,hwinfo__platform__architecture,hwinfo__platform__machine,hwinfo__platform__node,hwinfo__platform__platform,hwinfo__platform__processor,hwinfo__platform__release,hwinfo__platform__version,os_release__ansi_color,os_release__bug_report_url,os_release__cpe_name,os_release__home_url,os_release__id,os_release__id_like,os_release__name,os_release__platform_id,os_release__pretty_name,os_release__redhat_bugzilla_product,os_release__redhat_bugzilla_product_version,os_release__redhat_support_product,os_release__redhat_support_product_version,os_release__version,os_release__version_id,runtime_environment__cuda_version,runtime_environment__hardware__cpu_family,runtime_environment__hardware__cpu_model,runtime_environment__operating_system__name,runtime_environment__operating_system__version,runtime_environment__python_version,stdout__@parameters__device,stdout__@parameters__dtype,stdout__@parameters__matrix_size,stdout__@parameters__mini_batch,stdout__@parameters__reps,stdout__@result__elapsed,stdout__@result__rate,stdout__component,stdout__name,stdout__tensorflow_aicoe_buildinfo,stdout__tensorflow_upstream_buildinfo,usage__ru_idrss,usage__ru_inblock,usage__ru_isrss,usage__ru_ixrss,usage__ru_majflt,usage__ru_maxrss,usage__ru_minflt,usage__ru_msgrcv,usage__ru_msgsnd,usage__ru_nivcsw,usage__ru_nsignals,usage__ru_nswap,usage__ru_nvcsw,usage__ru_oublock,usage__ru_stime,usage__ru_utime,requirements_locked___meta__sources,requirements_locked___meta__requires__python_version,requi

## Create inspection results summary

In [ ]:
report_results, _ = inspection_runs_summary.produce_summary_report(inspections_df=inspections_df)

### Hardware

In [37]:
report_results["hardware"]['platform'].head()

,hwinfo__platform__architecture,hwinfo__platform__machine,hwinfo__platform__node,hwinfo__platform__platform,hwinfo__platform__processor,hwinfo__platform__release,hwinfo__platform__version
0,nan,x86_64,inspection-tf-dm-rw-000b1ef5-2959926406,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
1,nan,x86_64,inspection-tf-dm-rw-00590914-1708451517,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
2,nan,x86_64,inspection-tf-dm-rw-0175fdc8-874487295,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
3,nan,x86_64,inspection-tf-dm-rw-04fa3a4b-4099302384,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
4,nan,x86_64,inspection-tf-dm-rw-05f8f419-605987058,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-...,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020


In [24]:
report_results["hardware"]['processor']

,hwinfo__cpu_type__has_3dnow,hwinfo__cpu_type__has_3dnowext,hwinfo__cpu_type__has_Altivec,hwinfo__cpu_type__has_f00f_bug,hwinfo__cpu_type__has_fdiv_bug,hwinfo__cpu_type__has_mmx,hwinfo__cpu_type__has_sse,hwinfo__cpu_type__has_sse2,hwinfo__cpu_type__has_sse3,hwinfo__cpu_type__has_ssse3,hwinfo__cpu_type__is_32bit,hwinfo__cpu_type__is_64bit,hwinfo__cpu_type__is_AMD,hwinfo__cpu_type__is_AMD64,hwinfo__cpu_type__is_Alpha,hwinfo__cpu_type__is_Athlon64,hwinfo__cpu_type__is_AthlonHX,hwinfo__cpu_type__is_AthlonK6,hwinfo__cpu_type__is_AthlonK6_2,hwinfo__cpu_type__is_AthlonK6_3,hwinfo__cpu_type__is_AthlonK7,hwinfo__cpu_type__is_AthlonMP,hwinfo__cpu_type__is_Celeron,hwinfo__cpu_type__is_Core2,hwinfo__cpu_type__is_EV4,hwinfo__cpu_type__is_EV5,hwinfo__cpu_type__is_EV56,hwinfo__cpu_type__is_Hammer,hwinfo__cpu_type__is_Intel,hwinfo__cpu_type__is_Itanium,hwinfo__cpu_type__is_Nocona,hwinfo__cpu_type__is_Opteron,hwinfo__cpu_type__is_PCA56,hwinfo__cpu_type__is_Pentium,hwinfo__cpu_type__is_PentiumII,hwinfo__cpu_type__is_PentiumIII,hwinfo__cpu_type__is_PentiumIV,hwinfo__cpu_type__is_PentiumM,hwinfo__cpu_type__is_PentiumMMX,hwinfo__cpu_type__is_PentiumPro,hwinfo__cpu_type__is_Power,hwinfo__cpu_type__is_Power7,hwinfo__cpu_type__is_Power8,hwinfo__cpu_type__is_Power9,hwinfo__cpu_type__is_Prescott,hwinfo__cpu_type__is_XEON,hwinfo__cpu_type__is_Xeon,hwinfo__cpu_type__is_i386,hwinfo__cpu_type__is_i486,hwinfo__cpu_type__is_i586,hwinfo__cpu_type__is_i686,hwinfo__cpu_type__is_singleCPU
0,False,False,nan,nan,nan,True,True,True,True,True,False,True,False,False,nan,False,False,False,False,False,False,False,False,False,nan,nan,nan,False,True,nan,False,False,nan,False,False,False,False,False,False,False,nan,nan,nan,nan,False,True,True,nan,nan,False,True,False


In [25]:
report_results["hardware"]['flags']

,flag__3dnowprefetch,flag__abm,flag__adx,flag__aes,flag__apic,flag__arat,flag__avx,flag__avx2,flag__avx512bw,flag__avx512cd,flag__avx512dq,flag__avx512f,flag__avx512vl,flag__bmi1,flag__bmi2,flag__clflush,flag__clflushopt,flag__clwb,flag__cmov,flag__constant_tsc,flag__cpuid,flag__cx16,flag__cx8,flag__de,flag__ept,flag__erms,flag__f16c,flag__flexpriority,flag__fma,flag__fpu,flag__fsgsbase,flag__fxsr,flag__hle,flag__hypervisor,flag__ibpb,flag__ibrs,flag__invpcid,flag__invpcid_single,flag__lahf_lm,flag__lm,flag__mca,flag__mce,flag__md_clear,flag__mmx,flag__movbe,flag__mpx,flag__msr,flag__mtrr,flag__nopl,flag__nx,flag__ospke,flag__osxsave,flag__pae,flag__pat,flag__pcid,flag__pclmulqdq,flag__pdpe1gb,flag__pge,flag__pku,flag__pni,flag__popcnt,flag__pse,flag__pse36,flag__pti,flag__rdrand,flag__rdrnd,flag__rdseed,flag__rdtscp,flag__rep_good,flag__rtm,flag__sep,flag__smap,flag__smep,flag__ss,flag__ssbd,flag__sse,flag__sse2,flag__sse4_1,flag__sse4_2,flag__ssse3,flag__stibp,flag__syscall,flag__tpr_shadow,flag__tsc,flag__tsc_adjust,flag__tsc_deadline_timer,flag__tsc_known_freq,flag__tscdeadline,flag__vme,flag__vmx,flag__vnmi,flag__vpid,flag__x2apic,flag__xgetbv1,flag__xsave,flag__xsavec,flag__xsaveopt,flag__xtopology
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [26]:
report_results["hardware"]['ncpus']

,hwinfo__cpu_type__ncpus
0,64


In [27]:
report_results["hardware"]['info']

,hwinfo__cpu_info__brand_raw,runtime_environment__cuda_version,runtime_environment__hardware__cpu_family,runtime_environment__hardware__cpu_model
0,"Intel Xeon Processor (Skylake, IBRS)",nan,6,85


### Operating System

In [28]:
report_results["base_image"]['base_image']

,specification_base,os_release__name,os_release__version,os_release__version_id
0,quay.io/thoth-station/s2i-thoth-ubi8-py36,Red Hat Enterprise Linux,8.2 (Ootpa),8.2


In [29]:
report_results["base_image"]['number_cpus_run']

,run__requests__cpu
0,1


### Performance Indicators

In [30]:
report_results["pi"]['pi']

,script_sha256,batch_size,stdout__@parameters__device,stdout__@parameters__dtype,stdout__@parameters__matrix_size,stdout__@parameters__mini_batch,stdout__@parameters__reps,stdout__component,stdout__name
0,1ce2ae4cf0da06c8c55b99be8beb31cae7c6801bb99664...,1,cpu,float32,512,40,50,tensorflow,PiMatmul


### Software Stack

In [31]:
report_results["software_stack"]['requirements_locked'].head()

,requirements_locked___meta__sources,requirements_locked___meta__requires__python_version,requirements_locked___meta__hash__sha256,requirements_locked___meta__pipfile-spec,requirements_locked__default__tensorflow__version,requirements_locked__default__tensorflow__index,requirements_locked__default__tensorflow-estimator__version,requirements_locked__default__tensorflow-estimator__index,requirements_locked__default__astor__version,requirements_locked__default__astor__index,requirements_locked__default__wheel__version,requirements_locked__default__wheel__index,requirements_locked__default__protobuf__version,requirements_locked__default__protobuf__index,requirements_locked__default__google-pasta__version,requirements_locked__default__google-pasta__index,requirements_locked__default__six__version,requirements_locked__default__six__index,requirements_locked__default__tensorboard__version,requirements_locked__default__tensorboard__index,requirements_locked__default__requests__version,requirements_locked__default__requests__index,requirements_locked__default__opt-einsum__version,requirements_locked__default__opt-einsum__index,requirements_locked__default__numpy__version,requirements_locked__default__numpy__index,requirements_locked__default__google-auth__version,requirements_locked__default__google-auth__index,requirements_locked__default__chardet__version,requirements_locked__default__chardet__index,requirements_locked__default__google-auth-oauthlib__version,requirements_locked__default__google-auth-oauthlib__index,requirements_locked__default__keras-applications__version,requirements_locked__default__keras-applications__index,requirements_locked__default__termcolor__version,requirements_locked__default__termcolor__index,requirements_locked__default__idna__version,requirements_locked__default__idna__index,requirements_locked__default__certifi__version,requirements_locked__default__certifi__index,requirements_locked__default__cachetools__version,requirements_locked__default__cachetools__index,requirements_locked__default__h5py__version,requirements_locked__default__h5py__index,requirements_locked__default__pyasn1-modules__version,requirements_locked__default__pyasn1-modules__index,requirements_locked__default__werkzeug__version,requirements_locked__default__werkzeug__index,requirements_locked__default__gast__version,requirements_locked__default__gast__index,requirements_locked__default__wrapt__version,requirements_locked__default__wrapt__index,requirements_locked__default__requests-oauthlib__version,requirements_locked__default__requests-oauthlib__index,requirements_locked__default__markdown__version,requirements_locked__default__markdown__index,requirements_locked__default__keras-preprocessing__version,requirements_locked__default__keras-preprocessing__index,requirements_locked__default__oauthlib__version,requirements_locked__default__oauthlib__index,requirements_locked__default__grpcio__version,requirements_locked__default__grpcio__index,requirements_locked__default__setuptools__version,requirements_locked__default__setuptools__index,requirements_locked__default__absl-py__version,requirements_locked__default__absl-py__index,requirements_locked__default__urllib3__version,requirements_locked__default__urllib3__index,requirements_locked__default__scipy__version,requirements_locked__default__scipy__index,requirements_locked__default__pyasn1__version,requirements_locked__default__pyasn1__index,requirements_locked__default__rsa__version,requirements_locked__default__rsa__index
0,nan,3.6,c9e7f586cc7f527664b35f4581b3d5421512489efb2c89...,6,==2.1.0,pypi-org,==2.1.0,pypi-org,==0.6.2,pypi-org,==0.31.1,pypi-org,==3.11.0,pypi-org,==0.1.6,pypi-org,==1.13.0,pypi-org,==2.1.0,pypi-org,==2.21.0,pypi-org,==3.0.1,pypi-org,==1.17.3,pypi-org,==1.6.3,pypi-org,==2.3.0,pypi-org,==0.4.1,pypi-org,==1.0.8,pypi-org,==1.1.0,pypi-org,==2.0,pypi-org,==2018.8.13,pypi-org,==3.1.0,pypi-org,==2.10.0,pypi-org,==0.2.1,pypi-org,==0.14.1,pypi-org,==0.2.2,pypi-org,==1.11.1

In [32]:
python_packages_dataframe, _ = inspection.create_python_package_df(inspections_df=inspections_df)
python_packages_dataframe.head()

,tensorflow,tensorflow-estimator,astor,wheel,protobuf,google-pasta,six,tensorboard,requests,opt-einsum,numpy,google-auth,chardet,google-auth-oauthlib,keras-applications,termcolor,idna,certifi,cachetools,h5py,pyasn1-modules,werkzeug,gast,wrapt,requests-oauthlib,markdown,keras-preprocessing,oauthlib,grpcio,setuptools,absl-py,urllib3,scipy,pyasn1,rsa
0,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-3.0.1-pypi-org,numpy-1.17.3-pypi-org,google-auth-1.6.3-pypi-org,chardet-2.3.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-2.0-pypi-org,certifi-2018.8.13-pypi-org,cachetools-3.1.0-pypi-org,h5py-2.10.0-pypi-org,pyasn1-modules-0.2.1-pypi-org,werkzeug-0.14.1-pypi-org,gast-0.2.2-pypi-org,wrapt-1.11.1-pypi-org,requests-oauthlib-1.0.0-pypi-org,markdown-2.6.10-pypi-org,keras-preprocessing-1.1.0-pypi-org,oauthlib-2.0.5-pypi-org,grpcio-1.16.0-pypi-org,setuptools-23.2.1-pypi-org,absl-py-0.9.0-pypi-org,urllib3-1.10.4-pypi-org,scipy-1.4.1-pypi-org,pyasn1-0.4.5-pypi-org,rsa-3.4.1-pypi-org
1,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.32.1-pypi-org,protobuf-3.9.1-pypi-org,google-pasta-0.1.7-pypi-org,six-1.12.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.22.0-pypi-org,opt-einsum-2.3.2-pypi-org,numpy-1.16.5-pypi-org,google-auth-1.7.1-pypi-org,chardet-3.0.3-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-1.0-pypi-org,certifi-2017.4.17-pypi-org,cachetools-3.0.0-pypi-org,h5py-2.9.0-pypi-org,pyasn1-modules-0.2.3-pypi-org,werkzeug-0.11.15-pypi-org,gast-0.2.2-pypi-org,wrapt-1.11.1-pypi-org,requests-oauthlib-1.3.0-pypi-org,markdown-3.0-pypi-org,keras-preprocessing-1.1.0-pypi-org,oauthlib-3.0.1-pypi-org,grpcio-1.24.3-pypi-org,setuptools-41.6.0-pypi-org,absl-py-0.9.0-pypi-org,urllib3-1.16-pypi-org,scipy-1.4.1-pypi-org,pyasn1-0.4.2-pypi-org,rsa-3.2.3-pypi-org
2,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-2.3.2-pypi-org,numpy-1.16.5-pypi-org,google-auth-1.7.1-pypi-org,chardet-3.0.4-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-0.9-pypi-org,certifi-2017.7.27.1-pypi-org,cachetools-2.0.0-pypi-org,h5py-2.7.0-pypi-org,pyasn1-modules-0.2.5-pypi-org,werkzeug-0.15.1-pypi-org,gast-0.2.2-pypi-org,wrapt-1.11.1-pypi-org,requests-oauthlib-1.3.0-pypi-org,markdown-2.6.11-pypi-org,keras-preprocessing-1.1.0-pypi-org,oauthlib-3.1.0-pypi-org,grpcio-1.14.0-pypi-org,setuptools-41.3.0-pypi-org,absl-py-0.7.0-pypi-org,urllib3-0.3-pypi-org,scipy-1.4.1-pypi-org,pyasn1-0.4.4-pypi-org,rsa-3.1.4-pypi-org
3,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-2.3.2-pypi-org,numpy-1.17.0-pypi-org,google-auth-1.7.1-pypi-org,chardet-3.0.4-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-0.9-pypi-org,certifi-2017.7.27.1-pypi-org,cachetools-2.1.0-pypi-org,h5py-2.6.0-pypi-org,pyasn1-modules-0.2.5-pypi-org,werkzeug-0.15.1-pypi-org,gast-0.2.2-pypi-org,wrapt-1.11.2-pypi-org,requests-oauthlib-0.8.0-pypi-org,markdown-2.6.11-pypi-org,keras-preprocessing-1.1.0-pypi-org,oauthlib-1.0.0-pypi-org,grpcio-1.25.0-pypi-org,setuptools-41.3.0-pypi-org,absl-py-0.7.0-pypi-org,urllib3-0.3-pypi-org,scipy-1.4.1-pypi-org,pyasn1-0.4.4-pypi-org,rsa-3.1.4-pypi-org
4,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-py

In [33]:
final_dataframe = inspection.create_final_dataframe(
    inspections_df=inspections_df,
    include_statistics=True
)
final_dataframe.head()

,tensorflow,tensorflow-estimator,astor,wheel,protobuf,google-pasta,six,tensorboard,requests,opt-einsum,numpy,google-auth,chardet,google-auth-oauthlib,keras-applications,termcolor,idna,certifi,cachetools,h5py,pyasn1-modules,werkzeug,gast,wrapt,requests-oauthlib,markdown,keras-preprocessing,oauthlib,grpcio,setuptools,absl-py,urllib3,scipy,pyasn1,rsa,packages_list,sws_string,sws_hash_id,sws_hash_id_encoded,solver,os_name,os_version,python_interpreter,solver_string,solver_hash_id,base,cpu_brand,cpu_family,cpu_model,number_cpus,cuda_version,pi_name,pi_component,pi_sha256,elapsed_time,rate,inspection_document_id,identifier,standardized_identifier,start_datetime,end_datetime,total_duration,inspection_batch,statistical_parameter,std,std_error,mean,median,q1,q3,iqr,min,max,cov,covm,skew
0,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6-pypi-org,six-1.13.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.21.0-pypi-org,opt-einsum-3.0.1-pypi-org,numpy-1.17.3-pypi-org,google-auth-1.6.3-pypi-org,chardet-2.3.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-2.0-pypi-org,certifi-2018.8.13-pypi-org,cachetools-3.1.0-pypi-org,h5py-2.10.0-pypi-org,pyasn1-modules-0.2.1-pypi-org,werkzeug-0.14.1-pypi-org,gast-0.2.2-pypi-org,wrapt-1.11.1-pypi-org,requests-oauthlib-1.0.0-pypi-org,markdown-2.6.10-pypi-org,keras-preprocessing-1.1.0-pypi-org,oauthlib-2.0.5-pypi-org,grpcio-1.16.0-pypi-org,setuptools-23.2.1-pypi-org,absl-py-0.9.0-pypi-org,urllib3-1.10.4-pypi-org,scipy-1.4.1-pypi-org,pyasn1-0.4.5-pypi-org,rsa-3.4.1-pypi-org,"[tensorflow-2.1.0-pypi-org, tensorflow-estimat...",tensorflow-2.1.0-pypi-org<br>tensorflow-estima...,394b12484cc9d89aa303793fb4ad47ae4823f68779a6f5...,35,"[rhel, 8.2, 3.6]",rhel,8.2,3.6,rhel-8.2-py36,45249ee9cb34dc9242348683e441338c406e09e89c8c55...,quay.io/thoth-station/s2i-thoth-ubi8-py36,"Intel Xeon Processor (Skylake, IBRS)",6,85,1,None,PiMatmul,tensorflow,1ce2ae4cf0da06c8c55b99be8beb31cae7c6801bb99664...,691.177954,7.946493,inspection-tf-dm-rw-000b1ef5,tf-dm-rw,tf-dm-rw,2020-09-05 09:34:58.238825,2020-09-05 09:35:36.542039,38,1,elapsed_time,NaN,NaN,691.177954,691.177954,691.177954,691.177954,0.0,691.177954,691.177954,NaN,0.0,NaN
1,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.32.1-pypi-org,protobuf-3.9.1-pypi-org,google-pasta-0.1.7-pypi-org,six-1.12.0-pypi-org,tensorboard-2.1.0-pypi-org,requests-2.22.0-pypi-org,opt-einsum-2.3.2-pypi-org,numpy-1.16.5-pypi-org,google-auth-1.7.1-pypi-org,chardet-3.0.3-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,keras-applications-1.0.8-pypi-org,termcolor-1.1.0-pypi-org,idna-1.0-pypi-org,certifi-2017.4.17-pypi-org,cachetools-3.0.0-pypi-org,h5py-2.9.0-pypi-org,pyasn1-modules-0.2.3-pypi-org,werkzeug-0.11.15-pypi-org,gast-0.2.2-pypi-org,wrapt-1.11.1-pypi-org,requests-oauthlib-1.3.0-pypi-org,markdown-3.0-pypi-org,keras-preprocessing-1.1.0-pypi-org,oauthlib-3.0.1-pypi-org,grpcio-1.24.3-pypi-org,setuptools-41.6.0-pypi-org,absl-py-0.9.0-pypi-org,urllib3-1.16-pypi-org,scipy-1.4.1-pypi-org,pyasn1-0.4.2-pypi-org,rsa-3.2.3-pypi-org,"[tensorflow-2.1.0-pypi-org, tensorflow-estimat...",tensorflow-2.1.0-pypi-org<br>tensorflow-estima...,cc2a8a98803cc8377f4a224150b79ef4105fea5d7fda6b...,146,"[rhel, 8.2, 3.6]",rhel,8.2,3.6,rhel-8.2-py36,45249ee9cb34dc9242348683e441338c406e09e89c8c55...,quay.io/thoth-station/s2i-thoth-ubi8-py36,"Intel Xeon Processor (Skylake, IBRS)",6,85,1,None,PiMatmul,tensorflow,1ce2ae4cf0da06c8c55b99be8beb31cae7c6801bb99664...,595.723582,9.219781,inspection-tf-dm-rw-00590914,tf-dm-rw,tf-dm-rw,2020-09-04 14:13:51.158711,2020-09-04 14:14:22.235363,31,1,elapsed_time,NaN,NaN,595.723582,595.723582,595.723582,595.723582,0.0,595.723582,595.723582,NaN,0.0,NaN
2,tensorflow-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,astor-0.6.2-pypi-org,wheel-0.31.1-pypi-org,protobuf-3.11.0-pypi-org,google-pasta-0.1.6

## Plot results

In [39]:
import os

from typing import List

import pandas as pd
import plotly.graph_objects as go

from typing import Optional, Dict

pd.options.plotting.backend = "plotly"

from kaleido.scopes.plotly import PlotlyScope
scope = PlotlyScope()

if not os.path.exists("images"):
    os.mkdir("images")

_PERFORMANCE_QUANTITY = ["elapsed_time", "rate"]

_PERFORMANCE_QUANTITY_MAP = {"elapsed_time": "Elapsed Time [ms]", "rate": "Rate [GFLOPS]"}


class AmunInspectionsVisualization:
    """Class of methods used to create statistics from Amun Inspections Runs."""

    @staticmethod
    def create_inspection_3d_plot(
        plot_df: pd.DataFrame,
        varying_package: str,
        title_plot: str,
        quantity: Optional[str] = 'elapsed_time',
        image_name: Optional[str] = None,
    ):
        """Create inspection performance parameters plot in 3D.

        :param final_inspections_df: df for plots provided by `create_final_dataframe` or its subset.
        :param 
        """
        if quantity not in _PERFORMANCE_QUANTITY:
            _LOGGER.info(f"Only {_PERFORMANCE_QUANTITY} are accepted as quantity")
            return

        x_vector = [x[0] for x in plot_df[["solver_string"]].values]

        integer_y_encoded = [y[0] for y in plot_df[["sws_hash_id_encoded"]].values]

        z_vector = [z[0] for z in plot_df[[quantity]].values]

        trace1 = go.Scatter3d(
            x=x_vector,
            y=integer_y_encoded,
            z=z_vector,
            mode="markers",
            hovertext=[yc[0] for yc in plot_df[["sws_string"]].values],
            hoverinfo="text",
            marker=dict(
                size=8,
                color=z_vector,  # set color to an array/list of desired values
                colorscale="Viridis",  # choose a colorscale
                opacity=0.8,
                showscale=True,
            ),
        )

        data = [trace1]

        annotations = []
        c = 0

        for (x, y, z) in zip(x_vector, integer_y_encoded, z_vector):
            annotations.append(
                dict(
                    showarrow=False,
                    x=x,
                    y=y,
                    z=z,
                    text="".join(plot_df[varying_package].values[c]),
                    xanchor="left",
                    xshift=15,
                    opacity=0.7,
                )
            )
            c += 1

        margin = {"l": 200, "r": 100, "b": 100, "t": 100}

        layout = go.Layout(
            title=title_plot,
#             margin=margin,
            scene=dict(
                xaxis=dict(
                    title="Runtime Environment",
                    backgroundcolor="rgb(200, 200, 230)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white",
                    ),
                yaxis=dict(title="Software Stack ID",
                    backgroundcolor="rgb(230, 200,230)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white",
                    ),
                zaxis=dict(title=_PERFORMANCE_QUANTITY_MAP[quantity],
                    backgroundcolor="rgb(230, 230,200)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white"),
#                 annotations=annotations
            ),
            showlegend=True,
            legend=dict(orientation="h"),
        )
        fig = go.Figure(data=data, layout=layout)

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig
    
    
    @staticmethod
    def create_inspection_2d_plot(
        plot_df: pd.DataFrame,
        varying_package: str,
        title_plot: str,
        quantity: Optional[str] = 'elapsed_time',
        have_annotations: bool = False,
        image_name: Optional[str] = None,
    ):
        """Create inspection performance parameters plot in 2D.

        :param final_inspections_df: df for plots provided by `create_final_dataframe` or its subset.
        """
        integer_y_encoded = [y[0] for y in plot_df[["sws_hash_id_encoded"]].values]

        data = []
        annotations = []
        colour_counter = 0
        distance: float = 0
        name_component = varying_package

        subset_df = plot_df[plot_df["pi_component"] == varying_package]
        z_vector = [z[0] for z in subset_df[[quantity]].values]

        trace = go.Scatter(
            x=integer_y_encoded,
            y=z_vector,
            mode="markers",
            hovertext=[y[0] for y in subset_df[["sws_string"]].values],
            hoverinfo="text",
            marker=dict(
                size=12,
                color=z_vector,  # set color to an array/list of desired values
#                 colorscale=color_scales[colour_counter],  # choose a colorscale
                opacity=0.8,
                showscale=True,
                colorbar={"x": 1 + distance},
            ),
            name=f"solver=={plot_df['solver_string'].unique()[0]}",
        )

        data.append(trace)
        colour_counter += 1
        distance += 0.2

        layout = go.Layout(
            title=title_plot,
            xaxis=dict(title="Software Stack ID integer encoded"),
            yaxis=dict(title=_PERFORMANCE_QUANTITY_MAP[quantity]),
            showlegend=True,
            legend=dict(orientation="h", y=-0.3, yanchor="top"),
        )
        fig = go.Figure(data=data, layout=layout)

        if not image_name:
            image_name = title_plot

        with open(f"images/{image_name}.png", "wb") as f:
            f.write(scope.transform(fig, format="png"))

        return fig

In [ ]:
AmunInspectionsVisualization.create_inspection_3d_plot(plot_df=final_dataframe, varying_package="tensorflow", title_plot="TF==2.1.0 performance (3D plot)")

In [ ]:
AmunInspectionsVisualization.create_inspection_2d_plot(plot_df=final_dataframe, varying_package="tensorflow", title_plot="TF==2.1.0 performance (2D plot)")